# 06_SparkDataAnal.ipynb
TLC Trip Record Data
출처: https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [ ]:
# 2015-summary.json

In [2]:
df = spark.read.format('json')\
        .load('learning_spark_data/2015-summary.json')

In [3]:
df.count()

256

In [4]:
df.dtypes

[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint')]

In [5]:
df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



In [6]:
df.collect()

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344),
 Row(DEST_COUNTRY_NAME='Egypt', ORIGIN_COUNTRY_NAME='United States', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='India', count=62),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Grenada', count=62),
 Row(DEST_COUNTRY_NAME='Costa Rica', ORIGIN_COUNTRY_NAME='United States', count=588),
 Row(DEST_COUNTRY_NAME='Senegal', ORIGIN_COUNTRY_NAME='United States', count=40),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Sint Maarten', count=325),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Marshall Islands', count=39),
 

In [7]:
df.take(3)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Ireland', count=344)]

In [8]:
df.select('count').show(5)

+-----+
|count|
+-----+
|   15|
|    1|
|  344|
|   15|
|   62|
+-----+
only showing top 5 rows



In [9]:
df.select('DEST_COUNTRY_NAME').show(5)

+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
|    United States|
|            Egypt|
|    United States|
+-----------------+
only showing top 5 rows



In [11]:
# 도착 국가명 중복제거
df.select('DEST_COUNTRY_NAME').distinct().show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|            Anguilla|
|              Russia|
|            Paraguay|
|             Senegal|
|              Sweden|
|            Kiribati|
|              Guyana|
|         Philippines|
|            Djibouti|
|            Malaysia|
|           Singapore|
|                Fiji|
|              Turkey|
|                Iraq|
|             Germany|
|              Jordan|
|               Palau|
|Turks and Caicos ...|
|              France|
|              Greece|
+--------------------+
only showing top 20 rows



In [14]:
df1 = df.select('DEST_COUNTRY_NAME').distinct().cache()
df1.count()

132

In [15]:
# Row class를 이용한 단일 레코드 생성

from pyspark.sql import Row
myRow = Row('hello', None, 1, False)
myRow

<Row('hello', None, 1, False)>

In [16]:
# 새로운 컬럼 추가하기
from pyspark.sql.functions import expr

df3 = df.withColumn('withinCountry', expr('ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME')) # expr는 sql 표현식을 받아서 생성
df3

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint, withinCountry: boolean]

In [17]:
df3.show()

+--------------------+-------------------+-----+-------------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+--------------------+-------------------+-----+-------------+
|       United States|            Romania|   15|        false|
|       United States|            Croatia|    1|        false|
|       United States|            Ireland|  344|        false|
|               Egypt|      United States|   15|        false|
|       United States|              India|   62|        false|
|       United States|          Singapore|    1|        false|
|       United States|            Grenada|   62|        false|
|          Costa Rica|      United States|  588|        false|
|             Senegal|      United States|   40|        false|
|             Moldova|      United States|    1|        false|
|       United States|       Sint Maarten|  325|        false|
|       United States|   Marshall Islands|   39|        false|
|              Guyana|      United States|   64|       

In [23]:
df3.filter(df3.withinCountry == True).show( )

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [22]:
df3.filter(expr('withinCountry == True')).show()

+-----------------+-------------------+------+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|withinCountry|
+-----------------+-------------------+------+-------------+
|    United States|      United States|370002|         true|
+-----------------+-------------------+------+-------------+



In [28]:
# case when count < 10 => under, count > 10 => upper로 변환 (Category 컬럼 추가)
df4 = df.withColumn('Category', expr("CASE WHEN count<10 THEN 'under' WHEN count>=10 THEN 'upper' END"))
df4.show()

+--------------------+-------------------+-----+--------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|Category|
+--------------------+-------------------+-----+--------+
|       United States|            Romania|   15|   upper|
|       United States|            Croatia|    1|   under|
|       United States|            Ireland|  344|   upper|
|               Egypt|      United States|   15|   upper|
|       United States|              India|   62|   upper|
|       United States|          Singapore|    1|   under|
|       United States|            Grenada|   62|   upper|
|          Costa Rica|      United States|  588|   upper|
|             Senegal|      United States|   40|   upper|
|             Moldova|      United States|    1|   under|
|       United States|       Sint Maarten|  325|   upper|
|       United States|   Marshall Islands|   39|   upper|
|              Guyana|      United States|   64|   upper|
|               Malta|      United States|    1|   under|
|            A

In [ ]:
# DataFrame의 select(), where), filter() 트랜스포메이션
# show(), count() 액션

In [29]:
spark.stop()

In [30]:
# 집계함수

In [32]:
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [34]:
#emp_df, dept_df
emp_df  = spark.read.format('csv')\
            .option('header', 'true')\
            .option('inferSchema', 'true')\
            .load('learning_spark_data/emp.csv')
emp_df.printSchema()

root
 |-- empno: integer (nullable = true)
 |-- ename: string (nullable = true)
 |-- job: string (nullable = true)
 |-- mgr: integer (nullable = true)
 |-- hiredate: date (nullable = true)
 |-- sal: integer (nullable = true)
 |-- comm: integer (nullable = true)
 |-- deptno: integer (nullable = true)



In [35]:
dept_df  = spark.read.format('csv')\
            .option('header', 'true')\
            .option('inferSchema', 'true')\
            .load('learning_spark_data/dept.csv')
dept_df.printSchema()

root
 |-- deptno: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- loc: string (nullable = true)



In [36]:
emp_df.count()

15

In [37]:
dept_df.count()

4

In [38]:
# 컬럼명은 
emp_df.select('ENAME', 'DEPTNO').show()

+------+------+
| ENAME|DEPTNO|
+------+------+
| SMITH|    20|
| ALLEN|    30|
|  WARD|    30|
| JONES|    20|
|MARTIN|    30|
| BLAKE|    30|
| CLARK|    10|
| SCOTT|    20|
|  KING|    10|
|TURNER|    30|
| ADAMS|    20|
| JAMES|    30|
|  FORD|    20|
|MILLER|    10|
|  JACK|    70|
+------+------+



In [39]:
# filter()랑 동일함
emp_df.select('*').where('deptno = 20').show()

+-----+-----+-------+----+----------+----+----+------+
|empno|ename|    job| mgr|  hiredate| sal|comm|deptno|
+-----+-----+-------+----+----------+----+----+------+
| 7369|SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|
| 7566|JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7788|SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7876|ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|
| 7902| FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|
+-----+-----+-------+----+----------+----+----+------+



In [41]:
emp_df.selectExpr('count(*)').show()

+--------+
|count(1)|
+--------+
|      15|
+--------+



In [42]:
from pyspark.sql.functions import countDistinct

emp_df.select(countDistinct('job')).show()

+-------------------+
|count(DISTINCT job)|
+-------------------+
|                  5|
+-------------------+



In [43]:
from pyspark.sql.functions import approx_count_distinct
emp_df.select(approx_count_distinct('job', 0.1)).show()

+--------------------------+
|approx_count_distinct(job)|
+--------------------------+
|                         5|
+--------------------------+



In [51]:
# first, last, min, max, sum, avg => (expr: sql 문장 x), function으로 처리
# df.select(count('sal'))

from pyspark.sql.functions import first, last, min, max, sum, avg, count, mean

emp_df.select(
    count('sal').alias('count_sal'),
    first('sal').alias('first_sal'),
    last('sal').alias('last_sal'),
    min('sal').alias('min_sal'),
    max('sal').alias('max_sal'),
    sum('sal').alias('sum_sal'),
    avg('sal').alias('avg_sal')
).show()


+---------+---------+--------+-------+-------+-------+------------------+
|count_sal|first_sal|last_sal|min_sal|max_sal|sum_sal|           avg_sal|
+---------+---------+--------+-------+-------+-------+------------------+
|       15|      800|    3200|    800|   5000|  32225|2148.3333333333335|
+---------+---------+--------+-------+-------+-------+------------------+



In [52]:
emp_df.select(
    count('sal'),
    first('sal'),
    last('sal'),
    min('sal'),
    max('sal'),
    sum('sal'),
    avg('sal'),
    mean('sal')
).show()

+----------+----------+---------+--------+--------+--------+------------------+------------------+
|count(sal)|first(sal)|last(sal)|min(sal)|max(sal)|sum(sal)|          avg(sal)|          avg(sal)|
+----------+----------+---------+--------+--------+--------+------------------+------------------+
|        15|       800|     3200|     800|    5000|   32225|2148.3333333333335|2148.3333333333335|
+----------+----------+---------+--------+--------+--------+------------------+------------------+



In [48]:
emp_df.selectExpr('sum(distinct sal)').show()

+-----------------+
|sum(DISTINCT sal)|
+-----------------+
|            27975|
+-----------------+



In [ ]:
# total_sal / total_transaction, avg_sal, mean_sal

In [50]:
emp_df.selectExpr("sum(sal)/count(sal) as total_transaction").show()

# total_sal / total_transaction, avg_sal, mean_sal는 전부 동일하다

+------------------+
| total_transaction|
+------------------+
|2148.3333333333335|
+------------------+



In [59]:
emp_df.select(
    count('sal').alias('total_transaction'),
    sum('sal').alias('sum_sal'),
    avg('sal').alias('avg_sal'),
    mean('sal').alias('mean_sal')
).selectExpr(
    'round(sum_sal / total_transaction, 2) as manual_avg_sal',
    'round(avg_sal, 2) as avg_sal',
    'round(mean_sal, 2) as mean_sal'
).show()

+--------------+-------+--------+
|manual_avg_sal|avg_sal|mean_sal|
+--------------+-------+--------+
|       2148.33|2148.33| 2148.33|
+--------------+-------+--------+



In [57]:
# 그룹화
emp_df.groupBy('job').count(). show()

+---------+-----+
|      job|count|
+---------+-----+
|  ANALYST|    2|
| SALESMAN|    4|
|    CLERK|    5|
|  MANAGER|    3|
|PRESIDENT|    1|
+---------+-----+



In [60]:
# select job, 
#     count(job),
#     sum(sal)
# groupby job

emp_df.groupBy('job').agg(
    count('job').alias('quantity'),
    expr('count(job)'),
    sum('sal')
).show()

+---------+--------+----------+--------+
|      job|quantity|count(job)|sum(sal)|
+---------+--------+----------+--------+
|  ANALYST|       2|         2|    6000|
| SALESMAN|       4|         4|    5600|
|    CLERK|       5|         5|    7350|
|  MANAGER|       3|         3|    8275|
|PRESIDENT|       1|         1|    5000|
+---------+--------+----------+--------+



In [69]:
from pyspark.sql.functions import avg, expr, round

emp_df.groupBy('job').agg(
    round(avg('sal'), 2).alias('SAL_AVG'),
    round(expr('stddev(sal)'), 2).alias('SAL_STDEV')
).show()

+---------+-------+---------+
|      job|SAL_AVG|SAL_STDEV|
+---------+-------+---------+
|  ANALYST| 3000.0|      0.0|
| SALESMAN| 1400.0|   177.95|
|    CLERK| 1470.0|   984.63|
|  MANAGER|2758.33|   274.24|
|PRESIDENT| 5000.0|     NULL|
+---------+-------+---------+



In [68]:
# 급여 TOP 10 구하기
emp_df.orderBy(emp_df.sal.desc()).limit(10).show()

+-----+------+---------+----+----------+----+----+------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|
+-----+------+---------+----+----------+----+----+------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|
+-----+------+---------+----+----------+----+----+------+



In [73]:
# 윈도우 함수로 급여 TOP 10 구하기
from pyspark.sql.functions import desc, rank
from pyspark.sql.window import Window

windowspec = Window.orderBy(desc('sal'))
salAllRank = rank().over(windowspec)
salAllRank

Column<'RANK() OVER (ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [71]:
emp_df.withColumn('salary_rank', salAllRank).show(10)

+-----+------+---------+----+----------+----+----+------+-----------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|salary_rank|
+-----+------+---------+----+----------+----+----+------+-----------+
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|          1|
| 9292|  JACK|    CLERK|7782|1982-01-23|3200|NULL|    70|          2|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|          3|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|          3|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|          5|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|          6|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|          7|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|          8|
| 7844|TURNER| SALESMAN|7698|1981-09-08|1500|   0|    30|          9|
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|         10|
+-----+------+---------+----+----------+----+----+------+-----------+
only showing top 10 

In [77]:
# 직무별로 rank 작성
# Window, partitionBy()
# job_rank_df 작성

windowjob = Window.partitionBy('job').orderBy(desc('sal'))
job_rank_df = rank().over(windowjob)
job_rank_df

Column<'RANK() OVER (PARTITION BY job ORDER BY sal DESC NULLS LAST unspecifiedframe$())'>

In [78]:
emp_df.withColumn('salary_rank_by_job', job_rank_df).show(10)

+-----+------+-------+----+----------+----+----+------+------------------+
|empno| ename|    job| mgr|  hiredate| sal|comm|deptno|salary_rank_by_job|
+-----+------+-------+----+----------+----+----+------+------------------+
| 7788| SCOTT|ANALYST|7566|1987-04-19|3000|NULL|    20|                 1|
| 7902|  FORD|ANALYST|7566|1981-12-03|3000|NULL|    20|                 1|
| 9292|  JACK|  CLERK|7782|1982-01-23|3200|NULL|    70|                 1|
| 7934|MILLER|  CLERK|7782|1982-01-23|1300|NULL|    10|                 2|
| 7876| ADAMS|  CLERK|7788|1987-05-23|1100|NULL|    20|                 3|
| 7900| JAMES|  CLERK|7698|1981-12-03| 950|NULL|    30|                 4|
| 7369| SMITH|  CLERK|7902|1980-12-17| 800|NULL|    20|                 5|
| 7566| JONES|MANAGER|7839|1981-04-02|2975|NULL|    20|                 1|
| 7698| BLAKE|MANAGER|7839|1981-05-01|2850|NULL|    30|                 2|
| 7782| CLARK|MANAGER|7839|1981-06-09|2450|NULL|    10|                 3|
+-----+------+-------+---

In [76]:
emp_df.dtypes

[('empno', 'int'),
 ('ename', 'string'),
 ('job', 'string'),
 ('mgr', 'int'),
 ('hiredate', 'date'),
 ('sal', 'int'),
 ('comm', 'int'),
 ('deptno', 'int')]

In [88]:
# 부서별 순위
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

dept_window = Window.partitionBy('deptno').orderBy(emp_df.sal.desc())

emp_df.withColumn('SAL_RANK', rank().over(dept_window)).select('deptno', 'ename', 'sal', 'SAL_RANK').show()

+------+------+----+--------+
|deptno| ename| sal|SAL_RANK|
+------+------+----+--------+
|    10|  KING|5000|       1|
|    10| CLARK|2450|       2|
|    10|MILLER|1300|       3|
|    20| SCOTT|3000|       1|
|    20|  FORD|3000|       1|
|    20| JONES|2975|       3|
|    20| ADAMS|1100|       4|
|    20| SMITH| 800|       5|
|    30| BLAKE|2850|       1|
|    30| ALLEN|1600|       2|
|    30|TURNER|1500|       3|
|    30|  WARD|1250|       4|
|    30|MARTIN|1250|       4|
|    30| JAMES| 950|       6|
|    70|  JACK|3200|       1|
+------+------+----+--------+



In [89]:
# 누적 급여 sum('sal').over()
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

window_dept = Window.partitionBy('deptno').orderBy('sal')

emp_df.withColumn('DEPT_CUMULATIVE_SAL', sum('sal').over(window_dept)).show()

+-----+------+---------+----+----------+----+----+------+-------------------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|DEPT_CUMULATIVE_SAL|
+-----+------+---------+----+----------+----+----+------+-------------------+
| 7934|MILLER|    CLERK|7782|1982-01-23|1300|NULL|    10|               1300|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|               3750|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|               8750|
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|                800|
| 7876| ADAMS|    CLERK|7788|1987-05-23|1100|NULL|    20|               1900|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|               4875|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|              10875|
| 7902|  FORD|  ANALYST|7566|1981-12-03|3000|NULL|    20|              10875|
| 7900| JAMES|    CLERK|7698|1981-12-03| 950|NULL|    30|                950|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|       

In [90]:
# 부서별 누적급여
from pyspark.sql.functions import sum

emp_df.groupBy('deptno').agg(sum('sal').alias('TOTAL_SAL')).orderBy('deptno').show()

+------+---------+
|deptno|TOTAL_SAL|
+------+---------+
|    10|     8750|
|    20|    10875|
|    30|     9400|
|    70|     3200|
+------+---------+



In [91]:
# 부서별 평균급여와 직원 개별 급여 비교
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, round

dept_window = Window.partitionBy('deptno')

emp_df.withColumn(
    'DEPT_AVG_SAL',
    round(avg('sal').over(dept_window), 2)
).withColumn(
    'SAL_DIFF',
    round(emp_df.sal - avg('sal').over(dept_window), 2)
).select('deptno', 'ename', 'sal', 'DEPT_AVG_SAL', 'SAL_DIFF').show()

+------+------+----+------------+--------+
|deptno| ename| sal|DEPT_AVG_SAL|SAL_DIFF|
+------+------+----+------------+--------+
|    10| CLARK|2450|     2916.67| -466.67|
|    10|  KING|5000|     2916.67| 2083.33|
|    10|MILLER|1300|     2916.67|-1616.67|
|    20| SMITH| 800|      2175.0| -1375.0|
|    20| JONES|2975|      2175.0|   800.0|
|    20| SCOTT|3000|      2175.0|   825.0|
|    20| ADAMS|1100|      2175.0| -1075.0|
|    20|  FORD|3000|      2175.0|   825.0|
|    30| ALLEN|1600|     1566.67|   33.33|
|    30|  WARD|1250|     1566.67| -316.67|
|    30|MARTIN|1250|     1566.67| -316.67|
|    30| BLAKE|2850|     1566.67| 1283.33|
|    30|TURNER|1500|     1566.67|  -66.67|
|    30| JAMES| 950|     1566.67| -616.67|
|    70|  JACK|3200|      3200.0|     0.0|
+------+------+----+------------+--------+



In [94]:
# 부서별 집업별 소계
emp_df.groupBy('deptno', 'job').agg(count('*'),sum('sal'))\
        .orderBy('deptno', 'job').show()

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [93]:
emp_df.cube('deptno', 'job').agg(count('*'), sum('sal'))\
    .orderBy('deptno', 'job').show() # 평균급여, 최대급여, 최소급여

+------+---------+--------+--------+
|deptno|      job|count(1)|sum(sal)|
+------+---------+--------+--------+
|  NULL|     NULL|      15|   32225|
|  NULL|  ANALYST|       2|    6000|
|  NULL|    CLERK|       5|    7350|
|  NULL|  MANAGER|       3|    8275|
|  NULL|PRESIDENT|       1|    5000|
|  NULL| SALESMAN|       4|    5600|
|    10|     NULL|       3|    8750|
|    10|    CLERK|       1|    1300|
|    10|  MANAGER|       1|    2450|
|    10|PRESIDENT|       1|    5000|
|    20|     NULL|       5|   10875|
|    20|  ANALYST|       2|    6000|
|    20|    CLERK|       2|    1900|
|    20|  MANAGER|       1|    2975|
|    30|     NULL|       6|    9400|
|    30|    CLERK|       1|     950|
|    30|  MANAGER|       1|    2850|
|    30| SALESMAN|       4|    5600|
|    70|     NULL|       1|    3200|
|    70|    CLERK|       1|    3200|
+------+---------+--------+--------+



In [96]:
emp_dept_df = emp_df.join(dept_df, emp_df['deptno'] == dept_df['deptno'])
emp_dept_df.show()

+-----+------+---------+----+----------+----+----+------+------+----------+--------+
|empno| ename|      job| mgr|  hiredate| sal|comm|deptno|deptno|     dname|     loc|
+-----+------+---------+----+----------+----+----+------+------+----------+--------+
| 7369| SMITH|    CLERK|7902|1980-12-17| 800|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7499| ALLEN| SALESMAN|7698|1981-02-20|1600| 300|    30|    30|     SALES| CHICAGO|
| 7521|  WARD| SALESMAN|7698|1981-02-22|1250| 500|    30|    30|     SALES| CHICAGO|
| 7566| JONES|  MANAGER|7839|1981-04-02|2975|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7654|MARTIN| SALESMAN|7698|1981-09-28|1250|1400|    30|    30|     SALES| CHICAGO|
| 7698| BLAKE|  MANAGER|7839|1981-05-01|2850|NULL|    30|    30|     SALES| CHICAGO|
| 7782| CLARK|  MANAGER|7839|1981-06-09|2450|NULL|    10|    10|ACCOUNTING|NEW YORK|
| 7788| SCOTT|  ANALYST|7566|1987-04-19|3000|NULL|    20|    20|  RESEARCH|  DALLAS|
| 7839|  KING|PRESIDENT|NULL|1981-11-17|5000|NULL|    10|    10|A

In [97]:
join_df = emp_df.join(dept_df, on = 'deptno', how = 'inner')
join_df.select('ename', 'deptno', 'dname').show()

+------+------+----------+
| ename|deptno|     dname|
+------+------+----------+
| SMITH|    20|  RESEARCH|
| ALLEN|    30|     SALES|
|  WARD|    30|     SALES|
| JONES|    20|  RESEARCH|
|MARTIN|    30|     SALES|
| BLAKE|    30|     SALES|
| CLARK|    10|ACCOUNTING|
| SCOTT|    20|  RESEARCH|
|  KING|    10|ACCOUNTING|
|TURNER|    30|     SALES|
| ADAMS|    20|  RESEARCH|
| JAMES|    30|     SALES|
|  FORD|    20|  RESEARCH|
|MILLER|    10|ACCOUNTING|
+------+------+----------+



In [98]:
spark.stop()